In [17]:
from Data_and_Validation import *
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt

size = 9

data = create_data(size, 4, 1)
# data = [6, 29, 94, 35, 13, 103, -19, 0, 47]

print(data)

KWTA = KWTA_sorting(size, r = 4)
sorted_values, time_per_K = KWTA.parallel_sort(data, ascending=False)
print(time_per_K)



[31.44683135 27.19216036 35.70591117  8.96690009 13.68544778  4.71437139
 18.51279912 40.57039205 22.75585038]
KWTA # 1 state value: 4.5 update_direction: 1
KWTA # 1 state value: 8.5 update_direction: 1
KWTA # 1 state value: 12.5 update_direction: 1
KWTA # 1 state value: 16.5 update_direction: 1
KWTA # 1 state value: 20.5 update_direction: 1
KWTA # 1 state value: 24.5 update_direction: 1
KWTA # 1 state value: 28.5 update_direction: 1
KWTA # 1 state value: 32.5 update_direction: 1
KWTA # 1 state value: 36.5 update_direction: 1
KWTA # 1 state value: 36.5 update_direction: 0
State 1 converged at timestep = 10
KWTA # 2 state value: 4.5 update_direction: 1
KWTA # 2 state value: 8.5 update_direction: 1
KWTA # 2 state value: 12.5 update_direction: 1
KWTA # 2 state value: 16.5 update_direction: 1
KWTA # 2 state value: 20.5 update_direction: 1
KWTA # 2 state value: 24.5 update_direction: 1
KWTA # 2 state value: 28.5 update_direction: 1
KWTA # 2 state value: 32.5 update_direction: 1
KWTA # 2 sta

In [11]:
class KWTA_sorting():
    def __init__(self, input_size, initial_state = 0.5, r = 0.1, max_iter = 1000):
        self.input_size = input_size - 1 # number of neurons
        self.state = np.full(self.input_size, initial_state, dtype=float)
        self.r = r
        self.max_iterations = max_iter

    # heaviside adding
    def count_activations(self, input_vector, K):
        sum = 0
        for value in input_vector:
            sum = sum + self.step_function(value - self.state[K])
        return sum

    # Heaviside step activation: returns 1 if x > 0, otherwise 0.
    def step_function(self, x):
        return 1 if x > 0 else 0

    # Returns 1 if x > 0, -1 if x < 0, and 0 if x == 0.
    def signum(self, x):
        if x > 0:
            return 1 
        if x < 0:
            return -1 
        return 0
        

    def update_state(self, input_vector, K):
        # y(l+1)=y(l)+r⋅sgn(T− k=1∑N​S(y k​(l)))
        previous_state = self.state
        update_direction = self.signum(self.count_activations(input_vector, K) - (K + 1))
        self.state[K] = previous_state[K] + self.r * update_direction
        print("KWTA #", K + 1, "state value:", self.state[K], "update_direction:", update_direction)
        return update_direction == 0

    #the function call for using the sorting algorithm
    def parallel_sort(self, input_vector, ascending = True):
        total_time = 0
        time_per_K = []
        for K in range(self.input_size):
            t = 0
            steady = False
            state_at_t = []
            while not steady:
                t = t + 1
                total_time = total_time + 1
                steady = self.update_state(input_vector, K)
                state_at_t.append([t, self.state[K]])
                if t >= self.max_iterations:
                    print("max iterations reached")
                    return null, null
            print("State", K + 1, "converged at timestep =", t)
            time_per_K.append(state_at_t)
        

        print(self.state)
        
        S = []
        for n in range(self.input_size):
            Sn = []
            for value in input_vector:
                Sn.append(self.step_function(value - self.state[n]))
            S.append(Sn)
        S.append(np.full(self.input_size + 1, 1, dtype=int))

        sorting_matrix = [list(row) for row in zip(*S)]

        #sort the list by summing each row in the sorting matrix -> gives you a position in the final list
        orders = np.sum(sorting_matrix, axis=1) - 1

        if not ascending:
            orders = self.input_size - orders
        
        print( orders)
        print("Sorting matrix:")
        print(sorting_matrix)
        
        sorted_values = np.zeros(self.input_size + 1, dtype=int)
        for i, value in enumerate(input_vector):
            sorted_values[orders[i]] = value

        
        print(sorted_values)
        print("sorting time:", total_time)
        
        return sorted_values, time_per_K